# Testing out Deno with GeoTiff

Testing out the Deno jupyter kernel by using it to read in and render a geotiff file. The geotiff file is Sentinel-2 data taken from [Digital Earth Australia](https://maps.dea.ga.gov.au/) from somewhere on Bruny Island, Tasmania.

## Read GeoTiff Data

Reading in the file as bytes using `Deno.readFile`.
Using the `geotiff` library to parse the GeoTiff file.
The geotiff library wont actually load the bands into memory until we call readRasters.

# Testing Deno with Geotiff

In [8]:
import { fromArrayBuffer } from "npm:geotiff"
// Read all the file in
const fileBytes = await Deno.readFile("./sentinel_2_test.tiff")
const tiff = await fromArrayBuffer(fileBytes.buffer)
const image = await tiff.getImage(0)

// TODO - pretty print / format metadata
// image.fileDirectory.GDAL_METADATA


## Read in select bands

Load selected bands into the red, green and blue channels.

In [9]:
const [red, green, blue] = await image.readRasters({ samples: [3, 2, 1] })

## Setup stretch functions

I want to copy ENVI's default linear 2% stretch. This is a linear stretch that stretches the 2nd percentile to 98th percentile of the data to the 0-255 range.

In [10]:
import { quantile } from 'npm:d3-array';
import { scaleLinear } from 'npm:d3-scale';

// // Calculate 2nd and 98th percentiles
const blue2nd = quantile(blue.filter(d => d != -999), 0.02);
const blue98th = quantile(blue.filter(d => d != -999), 0.98);
const red2nd = quantile(red.filter(d => d != -999), 0.02);
const red98th = quantile(red.filter(d => d != -999), 0.98);
const green2nd = quantile(green.filter(d => d != -999), 0.02);
const green98th = quantile(green.filter(d => d != -999), 0.98);

// // Create new linear scales
const blueScale = scaleLinear([blue2nd, blue98th], [0, 255]).clamp(true);
const redScale = scaleLinear([red2nd, red98th], [0, 255]).clamp(true);
const greenScale = scaleLinear([green2nd, green98th], [0, 255]).clamp(true);

## Create image

Create an pixel array of the correct size and fill it with the stretched data.

In [12]:
const imageWidth = image.getWidth()
const imageHeight = image.getHeight()

// Each pixel is 4 bytes (r, g, b, a)
const pixelBytes = 4
const arrayBuffer = new ArrayBuffer(imageWidth * imageHeight * pixelBytes)
const pixels = new Uint8ClampedArray(arrayBuffer)

for (let i = 0; i < red.length; i++) {
  const pixelOffset = i * pixelBytes
  pixels[pixelOffset + 0] = redScale(red[i])     // red
  pixels[pixelOffset + 1] = greenScale(green[i]) // green
  pixels[pixelOffset + 2] = blueScale(blue[i])   // blue
  pixels[pixelOffset + 3] = 255                  // alpha
}

const imageData = new ImageData(pixels, imageWidth, imageHeight);
imageData

ImageData {
  data: Uint8ClampedArray(15270528) [
    4, 1, 12, 255, 4, 1, 12, 255, 5, 1, 23, 255,
    1, 4, 20, 255, 1, 0, 13, 255, 1, 0, 13, 255,
    2, 1, 14, 255, 2, 4, 23, 255, 2, 4, 23, 255,
    2, 2, 17, 255, 4, 2, 23, 255, 2, 0, 18, 255,
    2, 0, 18, 255, 1, 1, 13, 255, 1, 1, 16, 255,
    2, 0, 20, 255, 2, 0, 20, 255, 1, 2, 20, 255,
    3, 1, 24, 255, 3, 1, 24, 255, 2, 2, 18, 255,
    2, 2, 14, 255, 2, 2, 20, 255, 2, 2, 20, 255,
    2, 2, 20, 255,
    ... 15270428 more items
  ],
  width: 2093,
  height: 1824,
  colorSpace: "srgb"
}

: 

In [ ]:
// This was working before, now its not 🤷
import { createCanvas } from 'jsr:@gfx/canvas'
const canvas : HTMLCanvasElement = createCanvas(imageWidth, imageHeight)
const ctx = canvas.getContext('2d')!

ctx.putImageData(imageData, 0, 0);

canvas